In [1]:
import os, sys
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import torch

from torch.utils.data import DataLoader

KeyboardInterrupt: 

In [ ]:
sys.path.append('..')
from src.data.dataset import GWDataset, GWGridDataset, Normalize
from src.model.handler import ModelHandler
from src.model.neuralop.fno import FNO
from src.model.neuralop.losses import LpLoss, H1Loss

In [ ]:
base_data_dir = '/srv/scratch/z5370003/projects/data/groundwater/FEFLOW/coastal/variable_density/'
interpolated_data_dir = os.path.join(base_data_dir, 'interpolated')

In [ ]:
_mean = np.array([-0.5474])
_std = np.array([0.6562])
input_transform = Normalize(mean=_mean, std=_std)
output_transform = Normalize(mean=_mean, std=_std)

in_window_size = 5
out_window_size = 5
val_ratio = 0.3
batch_size = 32

fill_value = -1

In [ ]:
val_ds = GWGridDataset(data_path=interpolated_data_dir,
                         dataset='val', val_ratio=val_ratio,
                         in_window_size=in_window_size,
                         out_window_size=out_window_size,
                         input_transform=input_transform,
                         output_transform=output_transform,
                         fillval=fill_value)

val_dl = DataLoader(val_ds, batch_size=batch_size, 
                      shuffle=False, pin_memory=True)

len(val_dl)

In [ ]:
val_ds._data.shape[0]

In [ ]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(f"Found following device: {device}")

In [ ]:
# l2loss = LpLoss(d=3, p=2)
# h1loss = H1Loss(d=3)

# train_loss = h1loss
# eval_losses={'h1': h1loss, 'l2': l2loss}

# Model configuration
n_modes = (16, 16, 16)
in_channels = in_window_size
out_channels = out_window_size
hidden_channels = 64
projection_channels = 64
# scheduler_interval = 10

In [ ]:
model = FNO(n_modes=n_modes, in_channels=in_channels, 
            out_channels=out_channels,
            hidden_channels=hidden_channels, 
            projection_channels=projection_channels).double()

In [ ]:
results_path = '/srv/scratch/z5370003/projects/04_groundwater/variable_density/results/FNO/20250529_184905'
model_path = os.path.join(results_path, 'savedmodel_fno')
model.load_state_dict(torch.load(model_path, weights_only=True, map_location=device))

In [ ]:
model_handler = ModelHandler(model=model, device=device)

# Generate predictions
preds_mesh = np.array(model_handler.predict(val_dl))
preds_mesh = output_transform.inverse_transform(preds_mesh)

# Get targets
targets = model_handler.get_targets(val_dl)
targets = output_transform.inverse_transform(targets)

mask = targets == fill_value
preds_mesh[mask] = np.nan
targets[mask] = np.nan

In [12]:
points_data_dir = os.path.join(base_data_dir, 'all')

_mean = np.array([1523.10199582, 1729.62046887,   26.72577967,  316.5513978])
_std = np.array([569.13566635, 566.33636362,  14.90088159, 183.2160048 ])
input_transform = Normalize(mean=_mean, std=_std)

_mean = np.array([3.29223762e-01, 1.78798090e+04])
_std = np.array([2.17284490e-01, 1.51990336e+04])
output_transform = Normalize(mean=_mean, std=_std)

points_test_ds = GWDataset(points_data_dir, 'val', 
                     input_transform=input_transform, 
                     output_transform=output_transform,
                     val_ratio=0.3)


points_test_dl = DataLoader(points_test_ds, batch_size=10240, 
                      shuffle=False, pin_memory=True)
len(points_test_dl)

3434

In [13]:
n_points = points_test_ds.data.shape[0]//len(points_test_ds.datafiles)
points_test_ds.data.X.min(), points_test_ds.data.X.max()

(0.0, 2643.551370422647)

In [14]:
data = points_test_ds.data[['X', 'Y', 'Z', 'time (d)', 'head']].values

inputs = data[:, :4]
inputs[:, :3] = inputs[:, :3] + points_test_ds.origin
targets = data[:, 4:]

n_steps = inputs.shape[0]/n_points
ts = np.arange(n_steps, dtype=np.int64)
ts = ts[in_window_size:-out_window_size]

inputs = inputs.reshape((int(n_steps), -1, 4))
targets = targets.reshape((int(n_steps), -1, 1))

inputs = inputs[in_window_size:-out_window_size]
targets = targets[in_window_size:-out_window_size]

inputs.shape, targets.shape

((563, 61360, 4), (563, 61360, 1))

In [15]:
mlp_res_dir = '/srv/scratch/z5370003/projects/04_groundwater/variable_density/results/MLP/'
mlp_preds = np.load(os.path.join(mlp_res_dir, 'preds.npy'))

_mean = np.array([3.29223762e-01, 1.78798090e+04])
_std = np.array([2.17284490e-01, 1.51990336e+04])
output_transform = Normalize(mean=_mean, std=_std)

mlp_preds = output_transform.inverse_transform(mlp_preds).numpy()
mlp_preds = mlp_preds[:, 0]
mlp_preds = mlp_preds.reshape((int(n_steps), -1, 1))
mlp_preds = mlp_preds[in_window_size:-out_window_size]

mlp_preds.shape

(563, 61360, 1)

In [16]:
# 1. Define the grid
x_grid = val_ds.x_grid
y_grid = val_ds.y_grid
z_grid = val_ds.z_grid

# 3. Define query points
query_points = inputs[0, :, :3]

# Filter points inside the spacial bounds
# Filter x axis
x_filter_mask = (query_points[:, 0] < x_grid.max()) & (query_points[:, 0] > x_grid.min())
query_points = query_points[x_filter_mask]
targets = targets[:, x_filter_mask]
mlp_preds = mlp_preds[:, x_filter_mask]


# Filter y axis
y_filter_mask = (query_points[:, 1] < y_grid.max()) & (query_points[:, 1] > y_grid.min())
query_points = query_points[y_filter_mask]
targets = targets[:, y_filter_mask]
mlp_preds = mlp_preds[:, y_filter_mask]

# Filter z axis
z_filter_mask = (query_points[:, 2] < z_grid.max()) & (query_points[:, 2] > z_grid.min())
query_points = query_points[z_filter_mask]
targets = targets[:, z_filter_mask]
mlp_preds = mlp_preds[:, z_filter_mask]


In [17]:
preds_mesh = preds_mesh[:, 0].numpy()

In [18]:
from scipy.interpolate import RegularGridInterpolator

def query_values_from_mesh(preds_mesh, x_grid, y_grid, z_grid, query_points):
    
    # 2. Create the interpolator
    interpolator = RegularGridInterpolator((x_grid, y_grid, z_grid), preds_mesh)
    
    # 4. Query the interpolator
    queried_values = interpolator(query_points)

    return queried_values

In [19]:
from tqdm import tqdm

preds_points = []

for t in tqdm(range(preds_mesh.shape[0])):

    queried_values = query_values_from_mesh(preds_mesh[t], x_grid, y_grid,  z_grid, query_points)
    preds_points.append(queried_values[...,None])

preds_points = np.array(preds_points)

100%|██████████| 563/563 [00:04<00:00, 124.37it/s]


In [20]:
np.isnan(preds_points).all()

False

In [21]:
from sklearn.metrics import mean_squared_error, r2_score

In [22]:
mask = ~np.isnan(preds_points)

mean_squared_error(targets[mask].flatten(), preds_points[mask].flatten())

0.001922621593382303

In [23]:
mean_squared_error(targets.flatten(), mlp_preds.flatten())

0.12388200412842304

In [24]:
r2_score(targets[mask].flatten(), preds_points[mask].flatten())

0.9387202675959075

In [25]:
r2_score(targets.flatten(), mlp_preds.flatten())

-2.73651545011806

In [26]:
timesteps = np.arange(preds_mesh.shape[0])

for t in tqdm(timesteps):

    inputs_init = query_points
    targets_init = targets[t]
    fno_preds = preds_points[t]
    mlp_preds_init = mlp_preds[t]
    
    fig = plt.figure(figsize=(21, 6))
    ax1 = fig.add_subplot(1, 3, 1, projection='3d')
    ax2 = fig.add_subplot(1, 3, 2, projection='3d')
    ax3 = fig.add_subplot(1, 3, 3, projection='3d')
    
    im1 = ax1.scatter(inputs_init[:, 0], inputs_init[:, 1], inputs_init[:, 2], c=fno_preds[:, 0], alpha=0.5,
                      vmin=targets_init[:, 0].min(), vmax=targets_init[:, 0].max())
    ax1.set_title('Hydraulic-Head FNO')

    # im2 = ax2.scatter(inputs_init[:, 0], inputs_init[:, 1], inputs_init[:, 2], c=mlp_preds_init[:, 0], alpha=0.5,
    #                   vmin=targets_init[:, 0].min(), vmax=targets_init[:, 0].max())
    # ax2.set_title('Hydraulic-Head MLP')
    
    im2 = ax2.scatter(inputs_init[:, 0], inputs_init[:, 1], inputs_init[:, 2], c=targets_init[:, 0], alpha=0.5,
                      vmin=targets_init[:, 0].min(), vmax=targets_init[:, 0].max())
    ax2.set_title('Hydraulic-Head Targets')

    im3 = ax3.scatter(inputs_init[:, 0], inputs_init[:, 1], inputs_init[:, 2], c=fno_preds[:, 0]-targets_init[:, 0],
                      alpha=0.7, cmap='coolwarm')
    ax3.set_title('Error (Preds - Targets)')

    
    fig.colorbar(im1)
    fig.colorbar(im2)
    fig.colorbar(im3)

    os.makedirs('../results/FNO/point_preds_error/', exist_ok=True)
    
    fig.savefig(f'../results/FNO/point_preds_error/{t:d}.png', bbox_inches='tight')

    plt.clf()
    plt.close()
    # plt.show()
    # break

100%|██████████| 563/563 [26:58<00:00,  2.88s/it]


In [27]:
import cv2
import os

image_folder = '../results/FNO/point_preds_error/'
video_name = '../results/FNO/head_comparison_error.avi'

images = [img for img in os.listdir(image_folder) if img.endswith(".png")]
frame = cv2.imread(os.path.join(image_folder, images[0]))
height, width, layers = frame.shape

video = cv2.VideoWriter(video_name, 0, 4, (width, height))

for image in images:
    video.write(cv2.imread(os.path.join(image_folder, image)))

cv2.destroyAllWindows()
video.release()